# MRIO Workshop

In [361]:
import pandas as pd
import numpy as np

## 1.4 & 2 Load and clean data

### WIOT Data

In [364]:
lookup = pd.read_excel('Data/WIOT/wiot99_row_apr12.xlsx', index_col=None, header=None).iloc[2:6,4:].T.set_index([2,3])

In [365]:
lookup.head()

,,4,5
2,3,,
AtB,"Agriculture, Hunting, Forestry and Fishing",AUS,c1
C,Mining and Quarrying,AUS,c2
15t16,"Food, Beverages and Tobacco",AUS,c3
17t18,Textiles and Textile Products,AUS,c4
19,"Leather, Leather and Footwear",AUS,c5


In [185]:
# function for WIOT Data

def import_wiot(year):
    file_loc = 'Data/WIOT/wiot' + str(year)[2:] + '_row_apr12.xlsx'
    
    # import data
    df = pd.read_excel(file_loc, index_col=None, header=None).iloc[4:,2:]
    df[1644].fillna('Total', inplace=True)

    #set columns
    c1 = df.iloc[0].tolist()
    c1[0] = 'country'
    c2 = df.iloc[1].tolist()
    c2[1] = 'sector'
    df.columns = [c1, c2]

    # set index
    df = df.drop([4, 5], axis=0).set_index([('country', np.nan), (np.nan, 'sector')])
    
    return(df)

In [271]:
year_list = []
for i in range(1995, 2012):
    year = i
    year_list.append(year)

In [272]:
wiot_list = []
for i in year_list:
    df = import_wiot(i)
    wiot_list.append(df)

In [273]:
array_list = []
for i in range(len(wiot_list)):
    a = np.array(wiot_list[i])
    array_list.append(a)

In [274]:
wiot_data = np.array(array_list)

In [275]:
type(wiot_data)

numpy.ndarray

In [276]:
wiot_data.shape

(17, 1443, 1641)

In [277]:
# define as Z for later use
Z = wiot_data

## 3. Load the extension data

### Energy

In [348]:
# function for Energy Data --> make array of vecotrs containing total energy use per year per sector

def import_energy(country):
    array_list = []
    file_loc = 'Data/EAS-EU/' + str(country) + '_EU_May12.xls'
    for year in range(1995, 2010):
        sheet = str(year)
        # import data
        df = pd.read_excel(file_loc, sheet_name=sheet, index_col=None).drop([(np.nan, np.nan)], axis=0)
        df['Y'+str(year)] = df['TOTAL']
        df = df[['Y'+str(year)]].T
        a = np.array(df)
        array_list.append(a)
    return(array_list)

In [349]:
country_list = ['AUS','AUT','BEL','BGR','BRA','CAN','CHN','CYP','CZE','DEU','DNK','ESP','EST','FIN','FRA','GBR','GRC','HUN','IDN','IND','IRL','ITA','JPN','KOR','LTU','LUX','LVA','MEX','MLT',
                'NLD','POL','PRT','ROU','RUS','SVK','SVN','SWE','TUR','TWN','USA','ROW']
array_list = []

for country in country_list:
    energy = import_energy(country)
    array_list.append(energy)
energy_data = np.array(array_list)

In [350]:
type(energy_data)

numpy.ndarray

In [351]:
energy_data.shape

(41, 15, 1, 39)

In [352]:
# define as F for later use
F = energy_data

### Air Quality

In [359]:
# function for Energy Data --> make array of vecotrs containing total energy use per year per sector

def import_air(country):
    array_list = []
    file_loc = 'Data/EAS-AIR/' + str(country) + '_AIR_May12.xls'
    for year in range(1995, 2010):
        sheet = str(year)
        # import data
        df = pd.read_excel(file_loc, sheet_name=sheet, index_col=None).drop([(np.nan, np.nan)], axis=0)
        df = df[['CO2', 'CH4', 'N2O']]
        # convert to kilotonnes
        df['CH4'] = df['CH4'] / 1000
        df['N2O'] = df['N2O'] / 1000
        a = np.array(df)
        array_list.append(a)
    return(df)

In [360]:
import_air('AUS')

,,CO2,CH4,N2O
"Agriculture, Hunting, Forestry and Fishing",secAtB,6652.840866,3109.522585,62.940137
Mining and Quarrying,secC,24083.360753,1356.673632,0.129959
"Food, Beverages and Tobacco",sec15t16,3275.212245,1.187600,0.512764
Textiles and Textile Products,sec17t18,361.340018,0.036506,0.009625
"Leather, Leather and Footwear",sec19,84.018429,0.005426,0.001199
Wood and Products of Wood and Cork,sec20,139.226746,0.059842,0.013687
"Pulp, Paper, Paper , Printing and Publishing",sec21t22,2084.786280,0.178146,0.106621
"Coke, Refined Petroleum and Nuclear Fuel",sec23,7511.626556,16.680296,0.164594
Chemicals and Chemical Products,sec24,7942.215745,0.722865,10.906609
Rubber and Plastics,sec25,77.009592,0.019562,0.005994
